#### This is a working file that has been translated to a function within api_call.py and is not necessary to run. Run merge_df.ipynb only to achieve success!

In [2]:
import pandas as pd
import requests
from pprint import pprint
import os

# variables
url = "http://api.worldbank.org/v2/"

In [3]:
# Compile the query URL
query_url = f"{url}countries?format=json"
print(query_url)

http://api.worldbank.org/v2/countries?format=json


In [4]:
#this variable stores the number of pages so we can iterrate over all the pages since there was a max page size of 50 results.
#later I found out that there is no max page size of 50 and I was so proud of this code that I just pretended there still was a max of 50.
get_pages = requests.get(query_url).json()[0]['pages']

In [5]:
all_pages_list = []
country_list = []
poverty_list = []
region = []
capital_city = []
lat_long = []
iso2code = []

#this for loop grabs all the data from all the pages and puts it in a list so you don't have to call the API again.
for page in range(get_pages):
    all_pages_list.append(requests.get(f'{query_url}&page={page+1}').json()[1])
# pprint(all_pages_list)

#this nested for loop goes to every "page" of the list 50 times and grabs the required information out of them
for y in range(len(all_pages_list)):
    for x in range(len(all_pages_list[y])):
        country_list.append(all_pages_list[y][x]['name'])
        poverty_list.append(all_pages_list[y][x]['incomeLevel']['value'])
        region.append(all_pages_list[y][x]['region']['value'])
        capital_city.append(all_pages_list[y][x]['capitalCity'])
        lat_long.append(f"{all_pages_list[y][x]['latitude']} , {all_pages_list[y][x]['longitude']}")
        iso2code.append(all_pages_list[y][x]['iso2Code'])
# country_list
# poverty_list
# region
# capital_city
# lat_long

In [6]:
#create dataframe from all derived worldbank info
worldbank_df = pd.DataFrame({"Country": country_list,"Country Code": iso2code,"Region": region, "Poverty Level":poverty_list,"Capital City":capital_city,"Lat/Long":lat_long})

#drop rows that are not countries
worldbank_df = worldbank_df[worldbank_df.Region != "Aggregates"]
worldbank_df = worldbank_df.reset_index(drop=True)
worldbank_df

,Country,Country Code,Region,Poverty Level,Capital City,Lat/Long
0,Aruba,AW,Latin America & Caribbean,High income,Oranjestad,"12.5167 , -70.0167"
1,Afghanistan,AF,South Asia,Low income,Kabul,"34.5228 , 69.1761"
2,Angola,AO,Sub-Saharan Africa,Lower middle income,Luanda,"-8.81155 , 13.242"
3,Albania,AL,Europe & Central Asia,Upper middle income,Tirane,"41.3317 , 19.8172"
4,Andorra,AD,Europe & Central Asia,High income,Andorra la Vella,"42.5075 , 1.5218"
5,United Arab Emirates,AE,Middle East & North Africa,High income,Abu Dhabi,"24.4764 , 54.3705"
6,Argentina,AR,Latin America & Caribbean,Upper middle income,Buenos Aires,"-34.6118 , -58.4173"
7,Armenia,AM,Europe & Central Asia,Upper middle income,Yerevan,"40.1596 , 44.509"
8,American Samoa,AS,East Asia & Pacific,Upper middle income,Pago Pago,"-14.2846 , -170.691"
9,Antigua and Barbuda,AG,Latin America & Caribbean,High income,Saint John's,"17.1175 , -61.8456"


In [7]:
worldbank_df['Poverty Value (1-4)'] = worldbank_df['Poverty Level']
worldbank_df['Poverty Value (1-4)'] = worldbank_df['Poverty Value (1-4)'].replace('High income', 4)
worldbank_df['Poverty Value (1-4)'] = worldbank_df['Poverty Value (1-4)'].replace('Upper middle income', 3)
worldbank_df['Poverty Value (1-4)'] = worldbank_df['Poverty Value (1-4)'].replace('Lower middle income', 2)
worldbank_df['Poverty Value (1-4)'] = worldbank_df['Poverty Value (1-4)'].replace('Low income', 1)
worldbank_df


# worldbank_df['Poverty Level 1-5'] = poverty_value    
# worldbank_df['Poverty Level'].value_counts()

,Country,Country Code,Region,Poverty Level,Capital City,Lat/Long,Poverty Value (1-4)
0,Aruba,AW,Latin America & Caribbean,High income,Oranjestad,"12.5167 , -70.0167",4
1,Afghanistan,AF,South Asia,Low income,Kabul,"34.5228 , 69.1761",1
2,Angola,AO,Sub-Saharan Africa,Lower middle income,Luanda,"-8.81155 , 13.242",2
3,Albania,AL,Europe & Central Asia,Upper middle income,Tirane,"41.3317 , 19.8172",3
4,Andorra,AD,Europe & Central Asia,High income,Andorra la Vella,"42.5075 , 1.5218",4
5,United Arab Emirates,AE,Middle East & North Africa,High income,Abu Dhabi,"24.4764 , 54.3705",4
6,Argentina,AR,Latin America & Caribbean,Upper middle income,Buenos Aires,"-34.6118 , -58.4173",3
7,Armenia,AM,Europe & Central Asia,Upper middle income,Yerevan,"40.1596 , 44.509",3
8,American Samoa,AS,East Asia & Pacific,Upper middle income,Pago Pago,"-14.2846 , -170.691",3
9,Antigua and Barbuda,AG,Latin America & Caribbean,High income,Saint John's,"17.1175 , -61.8456",4


In [8]:
worldbank_df.to_csv(os.path.join("worldbank.csv"), index=None)

In [9]:
requests.get(f'{query_url}&page={1}').json()[1]

[{'id': 'ABW',
  'iso2Code': 'AW',
  'name': 'Aruba',
  'region': {'id': 'LCN',
   'iso2code': 'ZJ',
   'value': 'Latin America & Caribbean '},
  'adminregion': {'id': '', 'iso2code': '', 'value': ''},
  'incomeLevel': {'id': 'HIC', 'iso2code': 'XD', 'value': 'High income'},
  'lendingType': {'id': 'LNX', 'iso2code': 'XX', 'value': 'Not classified'},
  'capitalCity': 'Oranjestad',
  'longitude': '-70.0167',
  'latitude': '12.5167'},
 {'id': 'AFG',
  'iso2Code': 'AF',
  'name': 'Afghanistan',
  'region': {'id': 'SAS', 'iso2code': '8S', 'value': 'South Asia'},
  'adminregion': {'id': 'SAS', 'iso2code': '8S', 'value': 'South Asia'},
  'incomeLevel': {'id': 'LIC', 'iso2code': 'XM', 'value': 'Low income'},
  'lendingType': {'id': 'IDX', 'iso2code': 'XI', 'value': 'IDA'},
  'capitalCity': 'Kabul',
  'longitude': '69.1761',
  'latitude': '34.5228'},
 {'id': 'AFR',
  'iso2Code': 'A9',
  'name': 'Africa',
  'region': {'id': 'NA', 'iso2code': 'NA', 'value': 'Aggregates'},
  'adminregion': {'id':

In [10]:
country_list

['Aruba',
 'Afghanistan',
 'Africa',
 'Angola',
 'Albania',
 'Andorra',
 'Andean Region',
 'Arab World',
 'United Arab Emirates',
 'Argentina',
 'Armenia',
 'American Samoa',
 'Antigua and Barbuda',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Burundi',
 'East Asia & Pacific (IBRD-only countries)',
 'Europe & Central Asia (IBRD-only countries)',
 'Belgium',
 'Benin',
 'Burkina Faso',
 'Bangladesh',
 'Bulgaria',
 'IBRD countries classified as high income',
 'Bahrain',
 'Bahamas, The',
 'Bosnia and Herzegovina',
 'Latin America & the Caribbean (IBRD-only countries)',
 'Belarus',
 'Belize',
 'Middle East & North Africa (IBRD-only countries)',
 'Bermuda',
 'Bolivia',
 'Brazil',
 'Barbados',
 'Brunei Darussalam',
 'Sub-Saharan Africa (IBRD-only countries)',
 'Bhutan',
 'Botswana',
 'Sub-Saharan Africa (IFC classification)',
 'Central African Republic',
 'Canada',
 'East Asia and the Pacific (IFC classification)',
 'Central Europe and the Baltics',
 'Europe and Central Asia (IFC classification